In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC/CD8_pos_T/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC/CD8_pos_T/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/2.seurat_pipeline.240506/RNA_count_matrix/CD8_pos_T.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/Signac.CD8_pos_T.240517/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Prepare background data

In [6]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 121581 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100028543-100029371      *
       [2]     chr1 100037588-100039284      *
       [3]     chr1 100046056-100047225      *
       [4]     chr1 100048090-100048290      *
       [5]     chr1 100048674-100048824      *
       ...      ...                 ...    ...
  [121577]     chrX     9941344-9941682      *
  [121578]     chrX     9941870-9942777      *
  [121579]     chrX     9988885-9989045      *
  [121580]     chrX     9995625-9996370      *
  [121581]     chrX     9997430-9997928      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [7]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100028543-100029371,77,50,64,52,66,48,6,63,50,29,⋯,43,50,57,51,84,41.49578,829,chr1,1475,0.89280397
chr1-100037588-100039284,103,77,100,56,82,161,134,120,92,157,⋯,92,59,102,107,92,58.92752,1697,chr1,5171,0.97176368
chr1-100046056-100047225,58,31,65,88,55,46,11,97,52,47,⋯,91,76,85,114,199,38.71795,1170,chr1,247,0.70548852
chr1-100048090-100048290,14,7,12,8,14,11,2,17,7,11,⋯,15,6,14,19,24,47.76119,201,chr1,29,0.20480996
chr1-100048674-100048824,9,10,4,10,12,5,1,19,4,8,⋯,8,8,15,14,17,42.38411,151,chr1,37,0.29933953
chr1-100051979-100052210,16,8,21,13,17,11,0,15,11,15,⋯,20,14,9,25,25,43.10345,232,chr1,29,0.20480996
chr1-100059059-100059298,16,9,17,11,15,14,2,22,11,12,⋯,13,11,18,17,42,40.83333,240,chr1,32,0.24472574
chr1-100063644-100064065,30,24,34,16,28,33,5,35,36,23,⋯,28,9,21,48,18,52.36967,422,chr1,35,0.27904031
chr1-100064460-100065619,72,46,60,70,81,123,4,115,47,51,⋯,51,47,104,85,172,43.44828,1160,chr1,101,0.56112386


Run Signac prediction

In [8]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [9]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 8783901 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100028543-100029371      * |         AGL
        [2]     chr1 100028543-100029371      * |       ALG14
        [3]     chr1 100028543-100029371      * |      CDC14A
        [4]     chr1 100028543-100029371      * |         DBT
        [5]     chr1 100028543-100029371      * |        DPH5
        ...      ...                 ...    ... .         ...
  [8783897]     chrX     9997430-9997928      * |       TBL1X
  [8783898]     chrX     9997430-9997928      * |      TCEANC
  [8783899]     chrX     9997430-9997928      * |      TMSB4X
  [8783900]     chrX     9997430-9997928      * |     TRAPPC2
  [8783901]     chrX     9997430-9997928      * |        WWC3
                          PeakName               PairName    zscore    pvalue
                       <character>            <character>

In [ ]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna,
                                 matrix.atac,
                                 meta.features,
                                 16)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

In [14]:
execution_time

Time difference of 11.43181 hours

Save results

In [15]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

Warning message in dir.create(dir.output, recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/Signac.CD8_pos_T.240517' already exists”


GRanges object with 8783901 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100028543-100029371      * |         AGL
        [2]     chr1 100037588-100039284      * |         AGL
        [3]     chr1 100046056-100047225      * |         AGL
        [4]     chr1 100048090-100048290      * |         AGL
        [5]     chr1 100048674-100048824      * |         AGL
        ...      ...                 ...    ... .         ...
  [8783897]     chrX   96764950-96765184      * |      NAP1L3
  [8783898]     chrX   97155878-97156150      * |      NAP1L3
  [8783899]     chrX   97563307-97564550      * |      NAP1L3
  [8783900]     chrX   97597846-97598284      * |      NAP1L3
  [8783901]     chrX   97790972-97791168      * |      NAP1L3
                          PeakName               PairName    zscore    pvalue
                       <character>            <character>

In [16]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100028543,100029371,829,*,AGL,chr1-100028543-100029371,chr1-100028543-100029371_AGL,0.45912536,0.32307207,K562
chr1,100037588,100039284,1697,*,AGL,chr1-100037588-100039284,chr1-100037588-100039284_AGL,0.20706107,0.41798108,K562
chr1,100046056,100047225,1170,*,AGL,chr1-100046056-100047225,chr1-100046056-100047225_AGL,-0.98211781,0.83697909,K562
chr1,100048090,100048290,201,*,AGL,chr1-100048090-100048290,chr1-100048090-100048290_AGL,1.04605795,0.14776714,K562
chr1,100048674,100048824,151,*,AGL,chr1-100048674-100048824,chr1-100048674-100048824_AGL,-0.58670684,0.72129969,K562
chr1,100051979,100052210,232,*,AGL,chr1-100051979-100052210,chr1-100051979-100052210_AGL,-0.17885751,0.57097521,K562
chr1,100059059,100059298,240,*,AGL,chr1-100059059-100059298,chr1-100059059-100059298_AGL,-0.34613575,0.63537965,K562
chr1,100063644,100064065,422,*,AGL,chr1-100063644-100064065,chr1-100063644-100064065_AGL,-0.26846423,0.60582900,K562
chr1,100064460,100065619,1160,*,AGL,chr1-100064460-100065619,chr1-100064460-100065619_AGL,-1.52367121,0.93620457,K562


In [17]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2                     Matrix_1.6-5                     
 [3] BSgenome.Hsapiens.UCSC.hg38_1.4.5 BSgenome_1.